<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=eb2da359987dd26c606eab0014a150153ad06f8e41302e4018203b3be326b9a4
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3713 sha256=924b03b44f81403ca8bf9722c01e7fe183cc30e78ae3fbb5a02e748158a144c4
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    # url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    # r = requests.get(url)
    # with open('./data/data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4272,サンドラッグ,9989,小売業
4273,サックスバー ホールディングス,9990,小売業
4274,ジェコス,9991,卸売業
4277,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3085]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[50749]の株価情報を取得できなかったため、スキップしました。
経過時間: 312.65783405303955秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1195000,740.500000
1,ＩＮＰＥＸ,1605,鉱業,9573400,2263.500000
2,安藤・間,1719,建設業,799300,1243.000000
3,東急建設,1720,建設業,527500,832.000000
4,大林組,1802,建設業,2512600,1366.500000
...,...,...,...,...,...
355,ＮＴＴデータグループ,9613,情報・通信業,2680000,2101.000000
356,ＳＣＳＫ,9719,情報・通信業,648800,2627.500000
357,ベネッセホールディングス,9783,サービス業,575600,1852.000000
358,ヤマダホールディングス,9831,小売業,4929800,462.700012


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1195000,740.500000
1,ＩＮＰＥＸ,1605,鉱業,9573400,2263.500000
2,安藤・間,1719,建設業,799300,1243.000000
3,東急建設,1720,建設業,527500,832.000000
4,大林組,1802,建設業,2512600,1366.500000
...,...,...,...,...,...
255,大阪瓦斯,9532,電気・ガス業,934700,2541.000000
256,ＮＴＴデータグループ,9613,情報・通信業,2680000,2101.000000
257,ＳＣＳＫ,9719,情報・通信業,648800,2627.500000
258,ベネッセホールディングス,9783,サービス業,575600,1852.000000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＩＮＰＥＸ,1605,鉱業,9573400,2263.500000
1,安藤・間,1719,建設業,799300,1243.000000
2,東急建設,1720,建設業,527500,832.000000
3,大林組,1802,建設業,2512600,1366.500000
4,清水建設,1803,建設業,3219500,1053.000000
...,...,...,...,...,...
200,大阪瓦斯,9532,電気・ガス業,934700,2541.000000
201,ＮＴＴデータグループ,9613,情報・通信業,2680000,2101.000000
202,ＳＣＳＫ,9719,情報・通信業,648800,2627.500000
203,ベネッセホールディングス,9783,サービス業,575600,1852.000000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＬＩＦＵＬＬ,2120,サービス業,508500,234.000000
1,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,3950400,725.299988
2,パーソルホールディングス,2181,サービス業,732600,2479.000000
3,システナ,2317,情報・通信業,1354400,278.000000
4,日本駐車場開発,2353,不動産業,2639900,202.000000
...,...,...,...,...,...
95,ＫＡＤＯＫＡＷＡ,9468,情報・通信業,507500,3051.000000
96,イーレックス,9517,電気・ガス業,851100,789.000000
97,レノバ,9519,電気・ガス業,1048300,1155.000000
98,エイチ・アイ・エス,9603,サービス業,940900,1779.000000


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ＬＩＦＵＬＬ,2120,サービス業,508500,234.000000
1,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,3950400,725.299988
2,パーソルホールディングス,2181,サービス業,732600,2479.000000
3,日本駐車場開発,2353,不動産業,2639900,202.000000
4,カカクコム,2371,サービス業,1420200,1543.500000
5,ベネフィット・ワン,2412,サービス業,705200,1085.500000
6,エムスリー,2413,サービス業,2337000,2746.000000
7,博報堂ＤＹホールディングス,2433,サービス業,1506500,1263.000000
8,エスプール,2471,サービス業,604600,469.000000
9,円谷フィールズホールディングス,2767,卸売業,1584900,2143.000000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4267,タキヒヨー,9982,卸売業
4270,蔵王産業,9986,卸売業
4275,ヤマザワ,9993,小売業
4276,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[3954]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6647]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7957]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8072]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9049]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9720]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,2716900,473.0
1,クルーズ,2138,情報・通信業,411100,958.0
2,クックパッド,2193,サービス業,322400,145.0
3,第一屋製パン,2215,食料品,258800,497.0
4,クオンタムソリューションズ,2338,情報・通信業,463400,1870.0
...,...,...,...,...,...
61,アコム,8572,その他金融業,1665600,358.0
62,九州リースサービス,8596,その他金融業,209700,1009.0
63,トレイダーズホールディングス,8704,証券、商品先物取引業,206400,738.0
64,あかつき本社,8737,証券、商品先物取引業,241500,472.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,2716900,473.0
1,クルーズ,2138,情報・通信業,411100,958.0
2,第一屋製パン,2215,食料品,258800,497.0
3,クオンタムソリューションズ,2338,情報・通信業,463400,1870.0
4,ギグワークス,2375,サービス業,812000,339.0
5,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,210500,646.0
6,パレモ・ホールディングス,2778,小売業,219000,203.0
7,アップルインターナショナル,2788,卸売業,746200,521.0
8,ポラリス・ホールディングス,3010,サービス業,468100,168.0
9,オーミケンシ,3111,繊維製品,278300,367.0


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,2716900,473.0
1,第一屋製パン,2215,食料品,258800,497.0
2,クオンタムソリューションズ,2338,情報・通信業,463400,1870.0
3,ギグワークス,2375,サービス業,812000,339.0
4,オーミケンシ,3111,繊維製品,278300,367.0
5,ジェイ・エスコムホールディングス,3779,情報・通信業,2828400,236.0
6,パス,3840,情報・通信業,399800,156.0
7,Ｓｐｅｅｅ,4499,情報・通信業,753000,3815.0
8,日本高周波鋼業,5476,鉄鋼,458700,700.0
9,千代田化工建設,6366,建設業,908400,389.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,クックパッド,2193,サービス業,322400,145.0
1,クシム,2345,情報・通信業,301100,292.0
2,出前館,2484,情報・通信業,815500,359.0
3,セリア,2782,小売業,325600,2189.0
4,ＪＦＬＡホールディングス,3069,小売業,501600,171.0
5,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,285100,1076.0
6,セルシス,3663,情報・通信業,281500,710.0
7,ｅｎｉｓｈ,3667,情報・通信業,4120300,215.0
8,ディー・エル・イー,3686,情報・通信業,242900,247.0
9,フライトホールディングス,3753,情報・通信業,205800,401.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,クックパッド,2193,サービス業,322400,145.0
1,クシム,2345,情報・通信業,301100,292.0
2,出前館,2484,情報・通信業,815500,359.0
3,セリア,2782,小売業,325600,2189.0
4,ＪＦＬＡホールディングス,3069,小売業,501600,171.0
5,ｅｎｉｓｈ,3667,情報・通信業,4120300,215.0
6,ディー・エル・イー,3686,情報・通信業,242900,247.0
7,フライトホールディングス,3753,情報・通信業,205800,401.0
8,リミックスポイント,3825,情報・通信業,548900,198.0
9,Ａｂａｌａｎｃｅ,3856,電気機器,419500,3320.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

,銘柄名,コード,業種
45,ルーデン・ホールディングス,1400,建設業
46,エムビーエス,1401,建設業
55,Ｌｉｂ Ｗｏｒｋ,1431,建設業
60,フィット,1436,建設業
65,ニッソウ,1444,建設業
...,...,...,...
4091,プログリット,9560,サービス業
4092,グラッドキューブ,9561,サービス業
4093,ビジネスコーチ,9562,サービス業
4094,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,234000,632.0
1,ジーエヌアイグループ,2160,医薬品,3857200,1946.0
2,ウェッジホールディングス,2388,その他金融業,1750800,125.0
3,ベースフード,2936,食料品,931700,748.0
4,タスキ,2987,不動産業,640100,1404.0
...,...,...,...,...,...
82,ＩＮＦＯＲＩＣＨ,9338,サービス業,656500,4255.0
83,リニューアブル・ジャパン,9522,電気・ガス業,1292100,1076.0
84,Ｍ＆Ａ総研ホールディングス,9552,サービス業,500600,3300.0
85,マイクロアド,9553,サービス業,346800,2194.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,3857200,1946.0
1,ウェッジホールディングス,2388,その他金融業,1750800,125.0
2,ベースフード,2936,食料品,931700,748.0
3,タスキ,2987,不動産業,640100,1404.0
4,セキュアヴェイル,3042,情報・通信業,896400,305.0
5,サンワカンパニー,3187,小売業,257000,830.0
6,トラストホールディングス,3286,不動産業,454300,628.0
7,アプリックス,3727,情報・通信業,261700,212.0
8,クラウドワークス,3900,情報・通信業,232800,1459.0
9,シェアリングテクノロジー,3989,情報・通信業,613800,687.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ベースフード,2936,食料品,931700,748.0
1,タスキ,2987,不動産業,640100,1404.0
2,セキュアヴェイル,3042,情報・通信業,896400,305.0
3,サンワカンパニー,3187,小売業,257000,830.0
4,トラストホールディングス,3286,不動産業,454300,628.0
5,アプリックス,3727,情報・通信業,261700,212.0
6,アララ,4015,情報・通信業,613400,890.0
7,ＪＤＳＣ,4418,情報・通信業,792400,1294.0
8,スマレジ,4431,情報・通信業,555400,2523.0
9,サンバイオ,4592,医薬品,393300,663.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,234000,632.0
1,クリアル,2998,不動産業,292300,3900.0
2,海帆,3133,小売業,4546200,606.0
3,ＧＡ ｔｅｃｈｎｏｌｏｇｉｅｓ,3491,不動産業,250500,1167.0
4,Ａｉｍｉｎｇ,3911,情報・通信業,915600,351.0
5,グローバルウェイ,3936,情報・通信業,261600,191.0
6,プレイド,4165,情報・通信業,1010300,721.0
7,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,425900,1154.0
8,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,305900,300.0
9,ココナラ,4176,情報・通信業,1016000,342.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,234000,632.0
1,クリアル,2998,不動産業,292300,3900.0
2,Ａｉｍｉｎｇ,3911,情報・通信業,915600,351.0
3,グローバルウェイ,3936,情報・通信業,261600,191.0
4,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,425900,1154.0
5,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,305900,300.0
6,スパイダープラス,4192,情報・通信業,280500,608.0
7,エクサウィザーズ,4259,情報・通信業,665700,399.0
8,グローバルセキュリティエキスパート,4417,情報・通信業,202700,4480.0
9,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,923800,798.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[3954]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6647]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7957]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8072]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9049]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9720]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,2716900,473.0
1,ギグワークス,2375,サービス業,812000,339.0
2,出前館,2484,情報・通信業,815500,359.0
3,アップルインターナショナル,2788,卸売業,746200,521.0
4,ＪＦＬＡホールディングス,3069,小売業,501600,171.0
5,ｅｎｉｓｈ,3667,情報・通信業,4120300,215.0
6,ジェイ・エスコムホールディングス,3779,情報・通信業,2828400,236.0
7,リミックスポイント,3825,情報・通信業,548900,198.0
8,セプテーニ・ホールディングス,4293,サービス業,536300,378.0
9,Ｓｐｅｅｅ,4499,情報・通信業,753000,3815.0


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
3,アップルインターナショナル,2788,卸売業,746200,521.0
8,セプテーニ・ホールディングス,4293,サービス業,536300,378.0
10,日本精蝋,5010,石油・石炭製品,3291000,138.0
14,名村造船所,7014,輸送用機器,6694800,954.0
15,住信ＳＢＩネット銀行,7163,銀行業,1925200,1550.0
16,アトム,7412,小売業,598700,923.0
20,ラオックスホールディングス,8202,小売業,541400,248.0


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,3857200,1946.0
1,ウェッジホールディングス,2388,その他金融業,1750800,125.0
2,ベースフード,2936,食料品,931700,748.0
3,タスキ,2987,不動産業,640100,1404.0
4,セキュアヴェイル,3042,情報・通信業,896400,305.0
5,海帆,3133,小売業,4546200,606.0
6,Ａｉｍｉｎｇ,3911,情報・通信業,915600,351.0
7,シェアリングテクノロジー,3989,情報・通信業,613800,687.0
8,アララ,4015,情報・通信業,613400,890.0
9,プレイド,4165,情報・通信業,1010300,721.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,3857200,1946.0
1,ウェッジホールディングス,2388,その他金融業,1750800,125.0
5,海帆,3133,小売業,4546200,606.0
7,シェアリングテクノロジー,3989,情報・通信業,613800,687.0
9,プレイド,4165,情報・通信業,1010300,721.0
10,ヤプリ,4168,情報・通信業,585200,1311.0
11,ココナラ,4176,情報・通信業,1016000,342.0
16,ＢＡＳＥ,4477,情報・通信業,4915200,306.0
18,ＮＡＮＯ ＭＲＮＡ,4571,医薬品,3848100,217.0
19,メドレックス,4586,医薬品,3404800,236.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')